# RunnablePassthrough : 데이터를 효과적으로 전달하기

## 1. Set API Keys

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 2. RunnablePassthrough

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

chain = (
    PromptTemplate.from_template("{num}의 약수를 모두 알려줘.")
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | StrOutputParser()
)

chain을 invoke() 할 때 입력값의 형태는 딕셔너리 이어야 함

In [3]:
chain.invoke({"num" : 10})

'10의 약수는 1, 2, 5, 10입니다.'

langchain 라이브러리가 업데이트 되면서, Prompt Template에 변수가 1개라면 key 없이 값만 전달하는 것도 가능하다

In [4]:
chain.invoke(10)

'10의 약수는 1, 2, 5, 10입니다.'

구현하다보면 key값이 너무 많거나 기억하기 힘들어서 지정 없이 곧바로 값을 넣고 싶을 때가 존재함 -> 그럴 때 사용하는 것이 `RunnablePassthrough`

즉, 입력을 딕셔너리가 아닌 일반 값으로 수행하고 싶을 때, chain상에 딕셔너리를 작성학고 RunnablePassthrough 로 입력처리

`RunnablePassthrough`는 `runnable` 객체
- `runnable` 객체는 `invoke()` 와 같은 표준화된 인터페이스로 실행이 가능함

In [5]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({"num" : 10})

{'num': 10}

In [6]:
chain = (
    {"num" : RunnablePassthrough()}
    | PromptTemplate.from_template("{num}의 약수를 모두 알려줘.")
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | StrOutputParser()
)

chain.invoke(10)

'10의 약수는 1, 2, 5, 10입니다.'

`RunnablePassthrough.assign()`
- 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합침
- 즉, 원래 입력과 처리한 이후의 값을 모두 입력값으로 전달한다는 의미

> RunnablePassthrough에 입력된 값에 새로운 처리를 수행하고 새로운 입력값으로 매핑 가능

In [7]:
RunnablePassthrough.assign(new_num=lambda x : x["num"] * 3).invoke({"num" : 1})

{'num': 1, 'new_num': 3}